# Heuristic Search 

In [1]:
import gym
import minihack
import numpy as np
import matplotlib.pyplot as plt

from map_utils import *
from search import *
from graphics_utils import *

In [2]:
def solve_and_plt(env: gym.Env, heuristic: callable, precision: str, plt_width: Tuple[int, int], plt_height: Tuple[int, int], dynamic: bool = False, suppress = False):
    state = env.reset()
    game_map = state["chars"]
    color_map = state["colors"]
    pixel_map = state["pixel"]

    start = get_player_location(game_map)
    target = get_target_location(game_map)

    if dynamic:
        dynamic_path_finding(game_map, color_map, start, target, env, heuristic, precision=precision, graphics=True, pixel_map=pixel_map, suppress=suppress)
    else:
        path = a_star(game_map, color_map, start, target, heuristic, precision=precision)
        actions = actions_from_path(start, path[1:])
        render_actions(actions, env, pixel_map, plt_width, plt_height)

## 1. The Fully Observable Environment

The idea was to approach the problem in an increasing level of difficolty fashion, first working on some simpler heuristics and cost functions, up to reaching a more sofisticated level of complexity. 

We started by comparing the Euclidean Distance and the Chebyshev Distance, in order to get a better grasp on what the problem could have been approached with, and after a short analysis, we've found out that the Chebyshev looked exactly like what we were searching for, since it guaranteed 8 normal vectors that precisely represented what the actions of the agent are supposed to be.

In [ ]:
env = gym.make("MiniHack-Navigation-Custom-v0", observation_keys=("chars", "pixel", "colors"), des_file = "dat/simple_maze.des")
solve_and_plt(env, heuristic=euclidean_distance, precision="base", plt_width=(100, 270), plt_height=(500, 760))

In [ ]:
env = gym.make("MiniHack-Navigation-Custom-v0", observation_keys=("chars", "pixel", "colors"), des_file = "dat/simple_maze.des")
solve_and_plt(env, heuristic=chebyshev_distance, precision="base", plt_width=(100, 270), plt_height=(500, 760))

Initially, a simpler version of the problem was tackled. An A* algorithm was implemented that could find the optimal path in a map with trees.

In [ ]:
env = gym.make("MiniHack-Navigation-Custom-v0", observation_keys=["chars", "pixel", "colors"], des_file="dat/square_trees.des")
solve_and_plt(env, heuristic=chebyshev_distance, precision="base", plt_width=(100, 270), plt_height=(500, 760))

The algorithm also works on more intricate map configurations, that are not constrained to a square shape.

In [ ]:
env = gym.make("MiniHack-Navigation-Custom-v0", observation_keys=["chars", "pixel", "colors"], des_file="dat/maze_trees.des")
solve_and_plt(env, heuristic=chebyshev_distance, precision="base", plt_width=(100, 270), plt_height=(500, 760))

The problem was made more complex with the introduction of clouds and a "grid bug," a relatively weak monster in Nethack. This creature, matching the player's speed, is unable to ever catch up with the agent, adding a strategic element to the scenario. 

To effectively face this issue, a dynamic A* algorithm was implemented. This  solution recalculates the path based on whether the monster is visible or not. In scenarios where the monster is visible, the agent incurs no penalties traversing through clouded areas. Instead, in instances where the monster remains hidden, traversing through clouded regions carries a risk, as these areas could be secret hiding places for the lurking monster.


In [ ]:
env = gym.make("MiniHack-Navigation-Custom-v0", observation_keys=["chars", "pixel", "colors"], des_file="dat/grid_bug.des")
solve_and_plt(env, heuristic=chebyshev_distance, precision="base", plt_width=(100, 270), plt_height=(500, 760))

The final stage of the fully observable problem involved addressing the comprehensive challenge at hand. A diverse array of monsters was integrated into the scenario. 

At first, we just applied the strategy used so far to get to the target, but we soon found out that we may have needed some more sofistication.

In [ ]:
env = gym.make("MiniHack-HideNSeek-Mapped-v0", observation_keys=["chars", "pixel", "colors"])
solve_and_plt(env, heuristic=chebyshev_distance, precision="base", plt_width=(100, 270), plt_height=(500, 760))

The idea behind the improved strategy roots in two simple concepts:
- when the agent has the chance to see the monster in the map, it should keep itself further from it
- when the agent does not have a chance to get to know where the monster may be hiding itself, the agent must stay further from clouds as much as it might, for this very reason

In [ ]:
env = gym.make("MiniHack-HideNSeek-Mapped-v0", observation_keys=["chars", "pixel", "colors"])
solve_and_plt(env, heuristic=chebyshev_distance, precision="advanced", plt_width=(100, 270), plt_height=(500, 760), dynamic=True)

Let us give a brief discussion on the reasons behind the procedure rightfully working.

First, we will introduce the formulation of the Chebyshev Distance, i.e:
$$\text{let} \quad P = (p_1, p_2), \quad \text{and} \quad Q = (q_1, q_2)$$
$$d(P, Q) = \max(|p_1 - q_1|, |p_2 - q_2|)$$

The idea, as expressed before in more abstract terms, is to keep the agent as far as possible from the monster whenever it is visible from the map, this can be obtain by setting the $g$ function of the cost function $f$ in such a way that it weights cells closer to the monster more than cells further from it. 

Let us first define the cost function $f$:
$$f(P, T) := g(P, D) + h(P, T),$$
$$\text{where P is the neighbour position},$$
$$\text{D is the danger position},$$
$$\text{and T is the target position}$$

Note: the definition of the function $g$ actually needs to be taken in a more flexible way since, as we will see later, it will be having parameters of different types throughout all the instances of the problem (we will need $D$ to be considered as a set, and $g$ will have to be a function of a particular definition).

Anyway, what we mentioned above can be obtained by a simple manipulation of the function we used for the calculation of the distance between points in the plane, i.e. by simply calculating the reciprocal of the Chebyshev Distance between the position and the monsters itself. This gives us a function which is "rotation aware" (meaning it assigns the same value to positions falling in the radius of a circle (really squares to be precise, since the Chebyshev Distance actually is the $L_{\infty}$)).

$$\text{let} \quad \text{M be the monster's position},$$
$$\text{and} \quad \psi(P, M) := 
\begin{cases} 
    \frac{1}{d(P, M)}  & \text{if } d(P, M) > 0, \\
    maxcost & \text{if } d(P, M) = 0
\end{cases}, \quad \text{with} \quad maxcost = 10^{5}$$

$$\text{and} \quad h(P, T) := d(P, T)$$

We can obtain the cost function $f$ designed as described by setting $g = \psi$.

Notice that with $g$ we are comparing our neighbouring position $P$ with the monster position $M$, while with $h$ we are computing the exact distance (thanks to the Chebyshev Distance) between the neighbouring position $P$ and the target $T$.

Also, to be more precise, we implemented the above function with a slight variation, i.e. we split two cases from the one of the agent knowing the monster's position: one where $P$ is a cloud, and one where $P$ is not a cloud. As one might think, it can be advantageous to exploit clouds in order to run off the monster, but clouds have to be weighted in such a way that they must not incourage the agent to go against the monster in order to get in a cloud. To achieve this goal we could have implemented more sophisticated variations, but rather we chose a simple, yet effective approach (at least empirically), based on a very simple manipulation of the above function: 

$$\psi(P, M) := 
\begin{cases} 
    \frac{1}{d(P, M)}  & \text{if } d(P, M) > 0 \quad \text{and } P \notin Clouds, \\
    \left \lfloor \frac{1}{d(P, M)} \right \rfloor  & \text{if } d(P, M) > 0 \quad \text{and } P \in Clouds, \\
    maxcost & \text{if } d(P, M) = 0
\end{cases}, \quad \text{with} \quad maxcost = 10^{5}$$

The intuition behind this, is that the floor gives a very slight advantage to the cloud position in the cost function, precisely it will have an advantage $< 1$. This can be enough given that values won't be too big (by the size of the map), and hence we won't need large adjustments.

Now, let us make an observation about what we've seen so far: this only applies to cases where the agent does not know where the monster is located. In the other half of the cases, we need to make some more considerations: the agent must take care to get near clouds, since they really are ideal places for monsters to be hidden into. So, in order to "take care", we figured out a function to achieve this very reult.

$$\varphi(P, Clouds) := \sum_{C \in Clouds} \frac{1}{d(P, C)}$$

This function $\varphi$ can be interpreted as the "danger function", meaning that it gives a value to represent how much "dangerous" a position really is, basing the computation on the distance the position finds itself against each cloud in the map. To do so, we again exploit the reciprocal of the Chebyshev Distance, but this time we sum the amount of "danger" that each cloud project to the specific cell.

As before, we can use the original formulation, $f(P, T) := g(P, D) + h(P, T)$, to express the cost function, but this time $g = \varphi$.

$$f(P, T) := \sum_{C \in Clouds} \frac{1}{d(P, C)} + h(P, Q)$$

So, we can exploit the original formulation of $f$ to approach both the case where the monster is visible, and that of it being not in light, just by substituting $g$ with the function that suits the case.

Another important observation is that, in the case in which the agent does not know the monster's position, $g$ does not grow linearly as $h$ does (since it is of the form $\frac{1}{n}$ for the most part), hence this brings the conclusion that $h$ will have a bigger impact on most cases (i.e. each case where $h(P, T) > g(P, M)$, or to put it into words, every time the distance from the target is larger than the reciprocal of the distance from the monster, basically in most occasions). When we consider the case of $g = \varphi$, the reasoning actually changes a bit, since we are summing over the number of clouds in the map. In any case, this should not affect the algorithm in any particular way.

So, to conclude the discussion, we may interpret all this machinery as a filtering function: the first filter applies to positions that share the same value of $h$ (since that is the most weighted value), and amongst them the agent chooses the one that is less dangerous to be percolated.

A more involved analysis might be interesting to be carried on, but the choice to stop to this point is both of practical meaning and time constraints.

In any case, let us have a general discussion about the problem analysed. Since many of the monsters from the pool given by the MiniHack Environment, HideNSeek, are actually way too powerful to be played against with an empty inventory, we decided to only work on a small subset, one composed by only two monsters: Naga and the Giant. 

These two monsters appear to be pretty different from each other and this gives us a way to test the effectiveness of our approach on a level we'd like to.

#TODO expand discussion on monsters

In [ ]:
env = gym.make("MiniHack-Navigation-Custom-v0", des_file="dat/fully_observable_N.des", observation_keys=["chars", "pixel", "colors"])
solve_and_plt(env, heuristic=chebyshev_distance, precision="advanced", plt_width=(100, 270), plt_height=(500, 760), dynamic=True)

In [ ]:
win, loss, monsters_win, monsters_loss = evaluate_performance("MiniHack-Navigation-Custom-v0", dynamic_path_finding, des_file="dat/fully_observable_NH.des", evaluation_steps=500)

print(f"wins: {win}")
print(f"losses: {loss}")
print(f"beaten monsters: {monsters_win}")
print(f"monster that got us beaten: {monsters_loss}")

Let's have a look at the performance over Naga and the Giant alone

In [ ]:
# NAGA
win, loss, monsters_win, monsters_loss = evaluate_performance("MiniHack-Navigation-Custom-v0", dynamic_path_finding, des_file="dat/fully_observable_N.des", evaluation_steps=500)

print(f"wins: {win}")
print(f"losses: {loss}")
print(f"beaten monsters: {monsters_win}")
print(f"monster that got us beaten: {monsters_loss}")

In [ ]:
# GIANT
win, loss, monsters_win, monsters_loss = evaluate_performance("MiniHack-Navigation-Custom-v0", dynamic_path_finding, des_file="dat/fully_observable_H.des", evaluation_steps=500)

print(f"wins: {win}")
print(f"losses: {loss}")
print(f"beaten monsters: {monsters_win}")
print(f"monster that got us beaten: {monsters_loss}")

And also let's have a look at how the algoritmh performs on the entire pool (all the monsters from HideNSeek)

In [ ]:
win, loss, monsters_win, monsters_loss = evaluate_performance("MiniHack-HideNSeek-Mapped-v0", dynamic_path_finding, evaluation_steps=500)

print(f"wins: {win}")
print(f"losses: {loss}")
print(f"beaten monsters: {monsters_win}")
print(f"monster that got us beaten: {monsters_loss}")

#TODO statistics about monsters in the pool versions

In [ ]:
ml = [x for x in monsters_loss if x != None]
np.unique(ml, return_counts=True)

# TEST ON BLEND FEATURES

In [5]:
def solve_and_plt(env: gym.Env, heuristic: callable, precision: str, plt_width: Tuple[int, int], plt_height: Tuple[int, int], dynamic: bool = False, suppress = False):
    state = env.reset()
    game_map = state["chars"]
    color_map = state["colors"]
    pixel_map = state["pixel"]

    start = get_player_location(game_map)
    target = get_target_location(game_map)

    if dynamic:
        dpt_test(game_map, color_map, start, target, env, heuristic, precision=precision, graphics=True, pixel_map=pixel_map, suppress=suppress)
    else:
        path = a_star(game_map, color_map, start, target, heuristic, precision=precision)
        actions = actions_from_path(start, path[1:])
        render_actions(actions, env, pixel_map, plt_width, plt_height)

In [2]:
#NAGA
win, loss,noAction, monsters_win, monsters_loss = evaluate_performance("MiniHack-Navigation-Custom-v0", dpt_test, des_file="dat/fully_observable_N.des", evaluation_steps=500, graphics=False)

print(f"wins: {win}")
print(f"losses: {loss}")
print(f"noActions: {noAction}")
print(f"beaten monsters: {monsters_win}")
print(f"monster that got us beaten: {monsters_loss}")
print(f"winRate: {win/500}")

wins: 303
losses: 197
noActions: 0
beaten monsters: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', None, 'N', 'N', None, 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', None, 'N', 'N', 'N', 'N', 'N', 'N', 'N', None, 'N', 'N', None, 'N', 'N', 'N', 'N', None, 'N', 'N', 'N', 'N', 'N', None, 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', None, 'N', None, 'N', 'N', 'N', 'N', 'N', 'N', 'N', None, None, 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', None, 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', None, 'N', 'N', 'N', None, 'N', 'N', 'N', 'N', None, 'N', 'N', 'N', None, 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', None, None, 'N', 'N', 'N', 'N', 'N', None, None, None, 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', None, 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 